<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

In [1]:
import sys
import os


# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/PRÁCITCAS EXT/KUNVENO/KUNVENO_DIGITAL/'


pat_dir_checkpoints = path_absolute+'checkpoints'

El código se está ejecutando en Google Colab.
Mounted at /content/drive
Contenido de la carpeta en Google Drive:
['.git', 'Git_commnads.txt', 'README.md', 'checkpoints', 'icefall', '.gitignore', '.gitmodules', 'icefall_yes_no_dataset_recipe_with_CPU.ipynb']


In [2]:
!ls -la

total 33
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   399 Oct 11 18:53 .gitignore
-rw------- 1 root root    79 Oct 16 19:29 .gitmodules
drwx------ 2 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 16415 Oct 16 21:17 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root    17 Oct 11 16:57 README.md


In [3]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


### Install PyTorch and torchaudio

In [4]:
import torch
print(torch.__version__)

2.0.1+cu118


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [5]:
! pip install torchaudio==2.0.2

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [6]:
! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 MB 7.6 MB/s eta 0:00:00


Check that k2 was installed successfully:

In [7]:
! python3 -m k2.version


k2 version: 1.24.3
Build type: Release
Git SHA1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
Git date: Sat Jul 15 04:21:50 2023
Cuda used to build k2: 11.8
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.26.4
GCC version: 9.3.1
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -u

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [8]:
! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.9/731.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.7 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=429d1c5fa1503490ecb1953f5eee181b63b468afa64677ab41eb026d61f3d98a
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-2oird1zu
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-2oird1zu
  Resolved https://github.com/lhotse-speech/lhotse to commit ccfc5b2c27f5e11579a8ff15f5931b40095ee450
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lhotse: filename=lhotse-1.17.0.dev0+git.ccfc5b2.clean-py3-none-any.whl size=732388 sha256=6042be6512a7c3664bb16b75942c7eb11e17d507b7d60706c8b84493c05e6b2a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ma3nn2hp/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
Successfully built lhotse
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.17.0
    Uninstalling lhotse-1.17.0:
      Successfully uninstalled lhotse-1.17.0


### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [9]:
! pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


In [10]:
# ! git clone https://github.com/k2-fsa/icefall
# !git clone --recursive https://github.com/Jaime44/KUNVENO_DIGITAL.git


Now install dependencies of `icefall`:

In [11]:
!pip install typing-extensions==3.10.0.2


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.21 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
flax 0.7.4 requires typing-extensions>=4.2, but you have typing-extensions 3.10.0.2 which is incompatible.
librosa 0.10.1 requires typing-extensions>=4.1.1, but you have typing-extensions 3.10.0.2 which is incompatible.
polars 0.17.3 requires typing_extensions>=4.0.1;

In [12]:
! cd icefall && \
  pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [13]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0


In [14]:

!ls -la icefall/egs/yesno/ASR


total 32
drwx------ 2 root root 4096 Oct 16 20:56 data
drwx------ 2 root root 4096 Oct 12 19:24 download
drwx------ 2 root root 4096 Oct 11 18:53 local
drwx------ 2 root root 4096 Oct 12 19:19 portátil
-rw------- 1 root root 2066 Oct 12 19:52 prepare.sh
-rw------- 1 root root  335 Oct 12 19:51 README.md
lrw------- 1 root root   24 Oct 12 19:18 shared -> ../../../icefall/shared/
drwx------ 2 root root 4096 Oct 11 18:53 tdnn
drwx------ 2 root root 4096 Oct 16 20:40 tmp
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [15]:
!sudo chmod -R 755 icefall/egs/yesno/ASR/

In [16]:
!ls -la icefall/egs/yesno/ASR

total 32
drwx------ 6 root root 4096 Oct 16 20:56 data
drwx------ 3 root root 4096 Oct 12 19:24 download
drwx------ 2 root root 4096 Oct 11 18:53 local
drwx------ 3 root root 4096 Oct 12 19:19 portátil
-rwx------ 1 root root 2066 Oct 12 19:52 prepare.sh
-rwx------ 1 root root  335 Oct 12 19:51 README.md
lrw------- 1 root root   24 Oct 12 19:18 shared -> ../../../icefall/shared/
drwx------ 4 root root 4096 Oct 11 18:53 tdnn
drwx------ 3 root root 4096 Oct 16 20:40 tmp
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [17]:
!ls -la


total 63
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   399 Oct 11 18:53 .gitignore
-rw------- 1 root root    79 Oct 16 19:29 .gitmodules
drwx------ 3 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 47972 Oct 16 21:20 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root    17 Oct 11 16:57 README.md


In [18]:
!rm -r shared

rm: cannot remove 'shared': No such file or directory


In [19]:
!sudo ln -s ../../../icefall/shared/ icefall/egs/yesno/ASR/shared

ln: failed to create symbolic link 'icefall/egs/yesno/ASR/shared/shared': File exists


In [20]:
!rm -r icefall/icefall/transformer_lm/scaling.py


In [21]:
!sudo ln -s ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py icefall/icefall/transformer_lm/scaling.py

In [22]:
!ls -la icefall/icefall/transformer_lm/

total 69
-rw------- 1 root root 22349 Oct 12 19:52 attention.py
-rw------- 1 root root  5497 Oct 12 19:52 compute_perplexity.py
-rw------- 1 root root    20 Oct 12 19:52 dataset.py
-rw------- 1 root root 11234 Oct 12 19:52 encoder.py
-rw------- 1 root root  4925 Oct 12 19:52 export.py
-rw------- 1 root root     0 Oct 12 19:52 __init__.py
-rw------- 1 root root  3396 Oct 12 19:52 model.py
drwx------ 2 root root  4096 Oct 12 19:56 __pycache__
lrw------- 1 root root    65 Oct 16 21:21 scaling.py -> ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py
-rw------- 1 root root 17243 Oct 12 19:52 train.py


In [23]:
!ls -la icefall/egs/yesno/ASR

total 32
drwx------ 6 root root 4096 Oct 16 20:56 data
drwx------ 3 root root 4096 Oct 12 19:24 download
drwx------ 2 root root 4096 Oct 11 18:53 local
drwx------ 3 root root 4096 Oct 12 19:19 portátil
-rwx------ 1 root root 2066 Oct 12 19:52 prepare.sh
-rwx------ 1 root root  335 Oct 12 19:51 README.md
lrw------- 1 root root   24 Oct 12 19:18 shared -> ../../../icefall/shared/
drwx------ 4 root root 4096 Oct 11 18:53 tdnn
drwx------ 3 root root 4096 Oct 16 20:40 tmp
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [24]:
!rm -r icefall/egs/yesno/ASR/local/prepare_lang_fst.py


In [25]:
!ls -la icefall/egs/yesno/ASR/local

total 18
-rwx------ 1 root root  3562 Oct 12 19:51 compile_hlg.py
-rwx------ 1 root root  2834 Oct 12 19:51 compute_fbank_yesno.py
-rwx------ 1 root root 10925 Oct 12 19:51 prepare_lang.py


In [26]:
!sudo ln -s ../../../librispeech/ASR/local/prepare_lang_fst.py icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [27]:
!sudo chmod 755 icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [28]:
!ls -la icefall/egs/yesno/ASR/local

total 18
-rwx------ 1 root root  3562 Oct 12 19:51 compile_hlg.py
-rwx------ 1 root root  2834 Oct 12 19:51 compute_fbank_yesno.py
lrw------- 1 root root    50 Oct 16 21:21 prepare_lang_fst.py -> ../../../librispeech/ASR/local/prepare_lang_fst.py
-rwx------ 1 root root 10925 Oct 12 19:51 prepare_lang.py


In [29]:
!apt-get update && apt-get install dos2unix

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,005 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,347 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,270 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadco

In [30]:
# !dos2unix icefall/egs/yesno/ASR/prepare.sh
# !dos2unix icefall/icefall/shared/parse_options.sh

In [31]:
# !find icefall -type f -print0 | xargs -0 dos2unix

In [32]:
!ls -la icefall/icefall/shared/

total 46
-rw------- 1 root root  2892 Oct 12 19:52 convert-k2-to-openfst.py
-rw------- 1 root root 17663 Oct 12 19:52 make_kn_lm.py
-rw------- 1 root root 20805 Oct 12 19:52 ngram_entropy_pruning.py
-rw------- 1 root root  3657 Oct 12 19:52 parse_options.sh
lrw------- 1 root root    24 Oct 12 19:23 shared -> ../../../icefall/shared/


In [33]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd icefall/egs/yesno/ASR && \
  rm -rf data && \
  ./prepare.sh

2023-10-16 21:21:20 (prepare.sh:27:main) dl_dir: /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR/download
2023-10-16 21:21:20 (prepare.sh:30:main) Stage 0: Download data
2023-10-16 21:21:20 (prepare.sh:39:main) Stage 1: Prepare yesno manifest
2023-10-16 21:22:13 (prepare.sh:45:main) Stage 2: Compute fbank for yesno
2023-10-16 21:22:29,700 INFO [compute_fbank_yesno.py:65] Processing train
Extracting and storing features: 100% 90/90 [00:00<00:00, 133.72it/s]
2023-10-16 21:22:30,401 INFO [compute_fbank_yesno.py:65] Processing test
Extracting and storing features: 100% 30/30 [00:00<00:00, 201.94it/s]
2023-10-16 21:22:31 (prepare.sh:51:main) Stage 3: Prepare lang
2023-10-16 21:22:43,180 INFO [prepare_lang_fst.py:174] Building standard CTC topology
2023-10-16 21:22:43,183 INFO [prepare_lang_fst.py:183] Building L
2023-10-16 21:22:43,184 INFO [prepare_lang_fst.py:191] Building HL
2023-10-16 21:22:43,188 INFO [prepare_lang_fst.py:201] Skip building HLG


In [ ]:
!pwd


/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


## Training

In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
cd icefall/egs/yesno/ASR && \
./tdnn/train.py

In [ ]:
!ls -la


In [ ]:
!cd icefall/egs/yesno/ASR/tdnn/exp/tensorboard && \
tensorboard dev upload --logdir . --description "TDNN training for yesno with icefall"

## Decoding

In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

### Show the decoding result

In [ ]:
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

### Show the detailed WER

In [ ]:
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

# Pre-trained model

### Download the pre-trained model

In [ ]:
# ! cd /content/icefall/egs/yesno/ASR && \
#   mkdir tmp && \
#   cd tmp && \
#   git lfs install && \
#   git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cd tmp &&

In [ ]:
# ! sudo apt-get install git-lfs

In [ ]:
! sudo apt-get install tree

In [ ]:
# ! cd /content/icefall/egs/yesno/ASR && \
#   mkdir -p tmp && \
#   tree tmp

! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  tree tmp

In [ ]:
! sudo apt-get install sox

In [ ]:
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [ ]:
! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

## Inference with a pre-trained model


### View help information

In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py --help

### Decode a single sound file

In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

### Decode multiple sound files

In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav




**-----------------------------------------------------------------------------------------------------------------------------------------------------------**